In [ ]:
import os
import time
import openpyxl
import locale
import pandas as pd
from loguru import logger
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

In [37]:
class DatabaseConnection:
    def __init__(self):
        load_dotenv()
        self.DUSNEI_LOG_DIRECTORY = os.getenv('DUSNEI_LOG_DIRECTORY')
        self.LOG_FILE = os.path.join(self.DUSNEI_LOG_DIRECTORY, 'data.log')
        self.db_url = os.getenv('DUSNEI_URL')

    def initialize_logging(self):
        logger.add(
            sink= f'{self.DUSNEI_LOG_DIRECTORY}' + '/log_{time:YYYY-MM-DD}.log', 
            level='INFO', 
            rotation='1 day',
            format='{time:YYYY-MM-DD} | {function}: {message}'
        )

    def log_data(self):        
        for arquivo in os.listdir(self.DUSNEI_LOG_DIRECTORY):
            if arquivo.endswith('.log'):
                logger.info('Arquivo iniciado')

        
    def get_db_engine(self):
        try:
            self.db_url = os.getenv('DUSNEI_URL')
            engine = create_engine(self.db_url)
            # Test connection
            with engine.connect() as connection:
                logger.info('Conexão estabelecida!')
                pass
            logger.info('Banco de dados conectado!')
            return engine
        except exc.SQLAlchemyError as e:
            logger.info(f"Error: {e}")
            return None
        


In [47]:
class PendenciasClientes():
    def __init__(self):
        load_dotenv()
        self.path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_PENDENCIAS')
        self.conn = DatabaseConnection.get_db_engine(self)
        
    def pendencias_clientes_query(self, conn):
        query = (f"""
            SELECT
                pfin.pfin_transacao AS transacao,
                pfin.pfin_status AS status,
                pfin.pfin_datavcto AS datavcto,
                pfin.pfin_pger_conta AS pger_conta,
                pfin.pfin_unid_codigo AS unid_cod,
                pfin.pfin_vend_codigo AS vend_cod,
                pfin.pfin_codentidade AS entidade_cod,
                pfin.pfin_cnpjcpf AS cnpj,
                pfin.pfin_numerodcto AS nota,
                pfin.pfin_parcela AS parcelas,
                pfin.pfin_valor AS valor,
                pfin.pfin_nparcelas AS nparcelas,
                pger.pger_descricao AS descricao,
                pger.pger_tipo AS tipo,
                clie.clie_nome AS clie_nome,
                clie.clie_razaosocial AS razaosocial,
                clie.clie_vend_codigo AS clie_vend_cod,
                clie.clie_foneres AS fone_res,
                clie.clie_fonecel AS fone_cel,
                vend.vend_nome AS vend_nome,
                vend.vend_supe_codigo AS supe_cod,
                CONCAT(vend.vend_extra16, vend.vend_extra15) AS vend_cel,
                unid.unid_reduzido AS unidade,
                supe.supe_nome AS supe_nome
            FROM pendfin AS pfin
            INNER JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
            INNER JOIN unidades AS unid ON pfin.pfin_unid_codigo = unid.unid_codigo
            INNER JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
            INNER JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
            INNER JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
            WHERE pfin.pfin_status = 'P'
            AND pfin.pfin_datavcto < CURRENT_DATE
            AND pfin.pfin_pger_conta IN ('112131')
            ORDER BY pfin.pfin_datavcto DESC
        """)
        return pd.read_sql_query(query, conn)

    def save_to_excel(self):
        locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')
        df = self.pendencias_clientes_query(self.conn)
        
        wb = openpyxl.Workbook()
        ws = wb.active
        
        columns_title = [
            'Index','CodClie','RazaoSocial','CNPJ','Nota','Valor',
            'Vencimento','Fone','CodVend','Vendedor','Celular','Status','Hora'
        ]
        row_title = 1
        
        for col_num, column_title in enumerate(columns_title, 1):
            cell = ws.cell(row=row_title, column=col_num)
            cell.value = column_title

        for index in range(len(df)):
            cod_clie = df.loc[index, "entidade_cod"]
            razao_social = df.loc[index, "razaosocial"]
            cnpj = df.loc[index, "cnpj"]
            nota = df.loc[index, "nota"]
            valor = locale.currency(df.loc[index, "valor"], grouping=True)
            df.sort_values(by='datavcto', ascending=False)
            vencimento = format(df.loc[index, "datavcto"], "%Y/%m/%d")
            fone_res = df.loc[index, "fone_res"]
            fone_cel = df.loc[index, "fone_cel"]
            cod_vend = df.loc[index, "clie_vend_cod"]
            nome_vend = df.loc[index, "vend_nome"]
            cel_vend = df.loc[index, "vend_cel"]
            transacao = df.loc[index, "transacao"]
            status = "-"
            hora = "-"
            index = index + 1

            if fone_res == '':
                fone_res = fone_cel
            if fone_res == '':
                fone_res = '0'

            if cel_vend == '':
                cel_vend = '0'

            ws.cell(row=index+1, column=1).value = int(f'{index:.0f}')
            ws.cell(row=index+1, column=2).value = (f'{cod_clie:.0f}')
            ws.cell(row=index+1, column=3).value = (f'{razao_social}')
            ws.cell(row=index+1, column=4).value = (f'{cnpj}')
            ws.cell(row=index+1, column=5).value = (f'{nota}')
            ws.cell(row=index+1, column=6).value = (f'{valor}')
            ws.cell(row=index+1, column=7).value = (f'{vencimento}')
            ws.cell(row=index+1, column=8).value = (f'{fone_res}')
            ws.cell(row=index+1, column=9).value = (f'{cod_vend}')
            ws.cell(row=index+1, column=10).value = (f'{nome_vend}')
            ws.cell(row=index+1, column=11).value = (f'{cel_vend}')
            ws.cell(row=index+1, column=12).value = (f'{transacao}')
            ws.cell(row=index+1, column=13).value = (f'{status}')
            ws.cell(row=index+1, column=14).value = (f'{hora}')

        data_pasta = datetime.now().strftime("%Y-%m-%d")
        nome_arquivo = (f'pendencias-{data_pasta}')
        ws.title = nome_arquivo
        diretorio = f'{self.path_dados}/{data_pasta}'
        if not os.path.exists(diretorio):
                os.mkdir(diretorio)
        local_arquivo = os.path.join(f'{diretorio}/{nome_arquivo}.xlsx')
        wb.save(local_arquivo)
        
if __name__ == "__main__":
    teste = PendenciasClientes()
    teste.save_to_excel()

2023-12-12 11:34:38.109 | INFO     | __main__:get_db_engine:28 - Conexão estabelecida!
2023-12-12 11:34:38.110 | INFO     | __main__:get_db_engine:30 - Banco de dados conectado!
